# Clustering Crypto

In [1037]:
# Initial imports
import os
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [1038]:
# Load the crypto_data.csv dataset.
file = Path=('Resources/crypto_data.csv')

crypto_df = pd.read_csv(file, index_col=0)

crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [1039]:
# Look at the column data types
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [1040]:
# Find null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values.")

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column IsTrading has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 508 null values.
Column TotalCoinSupply has 0 null values.


In [1041]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df["IsTrading"] == True]

crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [1042]:
# Keep all the cryptocurrencies that have a working algorithm

# Printed each unique algorithm to get a list of the coin names
print(sorted((crypto_df["Algorithm"].unique())))

# Used pandas .notna to find all cryptocurrencies that have a working algorithm
pd.notna(crypto_df["Algorithm"])

['1GB AES Pattern Search', '536', 'Argon2', 'Argon2d', 'BLAKE256', 'Blake', 'Blake2S', 'Blake2b', 'C11', 'Cloverhash', 'Counterparty', 'CryptoNight', 'CryptoNight Heavy', 'CryptoNight Heavy X', 'CryptoNight-Lite', 'CryptoNight-V7', 'CryptoNight-lite', 'Cryptonight-GPU', 'Curve25519', 'DPoS', 'Dagger', 'Dagger-Hashimoto', 'ECC 256K1', 'Equihash', 'Equihash+Scrypt', 'Equihash1927', 'Ethash', 'Exosis', 'Green Protocol', 'Groestl', 'HMQ1725', 'HybridScryptHash256', 'IMesh', 'Jump Consistent Hash', 'Keccak', 'Leased POS', 'Lyra2RE', 'Lyra2REv2', 'Lyra2Z', 'M7 POW', 'Momentum', 'Multiple', 'NIST5', 'NeoScrypt', 'Ouroboros', 'PHI1612', 'POS 2.0', 'POS 3.0', 'PoS', 'Progressive-n', 'Proof-of-Authority', 'Proof-of-BibleHash', 'QUAIT', 'QuBit', 'Quark', 'QuarkTX', 'Rainforest', 'SHA-256', 'SHA-256 + Hive', 'SHA-256D', 'SHA-512', 'SHA3', 'SHA3-256', 'Scrypt', 'Scrypt-n', 'Semux BFT consensus', 'Shabal256', 'Skein', 'SkunkHash', 'SkunkHash v2 Raptor', 'Slatechain', 'Stanford Folding', 'T-Inside', 

42      True
365     True
404     True
611     True
808     True
        ... 
SERO    True
UOS     True
BDX     True
ZEN     True
XBC     True
Name: Algorithm, Length: 1144, dtype: bool

In [1043]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=["IsTrading"], inplace=True)

crypto_df

c:\Users\jlaro\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [1044]:
# Find null values

for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values.")

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 459 null values.
Column TotalCoinSupply has 0 null values.


In [1045]:
# Remove rows that have at least 1 null value.
crypto_drop_df = crypto_df.dropna()

# Check again for null values

for column in crypto_drop_df.columns:
    print(f"Column {column} has {crypto_drop_df[column].isnull().sum()} null values.")

crypto_drop_df

Column CoinName has 0 null values.
Column Algorithm has 0 null values.
Column ProofType has 0 null values.
Column TotalCoinsMined has 0 null values.
Column TotalCoinSupply has 0 null values.


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [1046]:
# Keep the rows where coins are mined.
crypto_df = crypto_drop_df[crypto_drop_df["TotalCoinsMined"] > 0]

crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [1047]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coinName_df = crypto_df[["CoinName"]]

coinName_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [1048]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
noCoin_crypto_df = crypto_df.copy()

noCoin_crypto_df.drop(columns=["CoinName"], axis=1, inplace=True)

noCoin_crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [1049]:
# Use get_dummies() to create variables for text features.
crypto_df = pd.get_dummies(noCoin_crypto_df, columns=["Algorithm", "ProofType"])

crypto_df.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1050]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(crypto_df)

print(crypto_scaled)

[[-0.11710817 -0.1528703  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.145009   -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561  4.48942416 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 ...
 [-0.09561336 -0.13217937 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11694817 -0.15255998 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11710536 -0.15285552 -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


### Deliverable 2: Reducing Data Dimensions Using PCA

In [1051]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)

pca_crypto = pca.fit_transform(crypto_scaled)

pca_crypto

array([[-0.33535695,  1.1292509 , -0.41659244],
       [-0.3186647 ,  1.12934393, -0.41681653],
       [ 2.30026983,  1.67841867, -0.51820829],
       ...,
       [ 0.3239914 , -2.36770681,  0.27920881],
       [-0.15429494, -2.08860333,  0.27802383],
       [-0.28249708,  0.78752714, -0.2512206 ]])

In [1052]:
# Create a DataFrame with the three principal components.
pca_crypto_df = pd.DataFrame(
    data = pca_crypto, columns = ["Principal Comp. 1", "Principal Comp. 2", "Principal Comp. 3"],
    index = crypto_df.index
)
pca_crypto_df

,Principal Comp. 1,Principal Comp. 2,Principal Comp. 3
42,-0.335357,1.129251,-0.416592
404,-0.318665,1.129344,-0.416817
1337,2.300270,1.678419,-0.518208
BTC,-0.148276,-1.323609,0.116456
ETH,-0.156529,-2.050366,0.272064
...,...,...,...
ZEPH,2.446149,0.936292,0.168627
GAP,-0.333398,1.129119,-0.416617
BDX,0.323991,-2.367707,0.279209
ZEN,-0.154295,-2.088603,0.278024


In [1053]:
# Checking how much information can be attributed to each principal component
# pc1 = .03%, pc2 = .02%, pc3 = .02% for a total of .07% of the information
pca.explained_variance_ratio_

array([0.02792946, 0.02138478, 0.02048205])

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [1054]:
NUM_THREADS = '3'
os.environ['OMP_NUM_THREADS'] = NUM_THREADS

# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculating inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state = 42)
    km.fit(pca_crypto_df)
    inertia.append(km.inertia_)

# Creating the actual elbow curve
elbow_data = {'k': k, 'inertia': inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = 'k', y = 'inertia', xticks = k, title = "Elbow Curve")

C:\Users\jlaro\AppData\Roaming\Python\Python37\site-packages\sklearn\cluster\_kmeans.py:1040: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [1055]:
# Initialize the K-Means model.
km_model = KMeans(n_clusters = 4, random_state = 42)

# Fit the model
km_model.fit(pca_crypto_df)

# Predict clusters
predictions = km_model.predict(pca_crypto_df)

predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [1056]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([noCoin_crypto_df, pca_crypto_df], axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coinName_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = km_model.labels_

# Giving the crytocurrencies abbreviations a column name of "Coin Abbr."
clustered_df.index.rename('Coin Abbr.', inplace = True)


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Principal Comp. 1,Principal Comp. 2,Principal Comp. 3,CoinName,Class
Coin Abbr.,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.335357,1.129251,-0.416592,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318665,1.129344,-0.416817,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.300270,1.678419,-0.518208,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148276,-1.323609,0.116456,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.156529,-2.050366,0.272064,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.161447,-1.082829,-0.012974,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.408962,1.278751,-0.349702,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.145113,-2.235906,0.291244,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154967,-2.050479,0.272045,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [1057]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,    
    x = "Principal Comp. 1",
    y = "Principal Comp. 2",
    z = "Principal Comp. 3",
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
    color = "Class",
    symbol = "Class",
    width = 800
)

fig.update_layout(legend = dict(x =0, y = 1))
fig.show()

In [1058]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [1059]:
# Print the total number of tradable cryptocurrencies.
print(f":. There are {clustered_df.CoinName.size} cryptocurrencies that are tradable .:")

:. There are 532 cryptocurrencies that are tradable .:


In [1060]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
crypto_scaler = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])

print(crypto_scaler)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [1061]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
combined_df = pd.DataFrame(
    data = crypto_scaler, columns = ["TotalCoinSupply", "TotalCoinsMined"],
    index = clustered_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
combined_df = pd.concat([combined_df, clustered_df["CoinName"]], axis = 1) 

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# Already done above? Check rubric.

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([combined_df, clustered_df["Class"]], axis = 1)


plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
Coin Abbr.,,,,
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0


In [1062]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
                    x = "TotalCoinsMined", 
                    y = "TotalCoinSupply",
                    by = "Class"         
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)